# Alice's $X$-Program

### Set up

In [1]:
import numpy as np
from random import sample, seed

Set seed for reproducibility.

In [2]:
seed(0)

This is a helper function to convert an integer $i$ to the list of length $n$ correspionding to the binary expansion of $i$. This enables us to efficiently sample a set of distinct binary vectors.

In [3]:
def binary_vector(i, n):
    return [int(b) for b in bin(i)[2:].zfill(n)]

### Construct the generator matrix for a quadratic residue code.

Select a prime $q$ such that $q+1$ is divisible by $8$.

In [4]:
q = 7

The rank of the code is $\frac{q+1}{2}$.

In [5]:
r = (q+1)//2

Find the quadratic residues modulo $q$.

In [6]:
residues = {(j**2) % q for j in range(1, q)}
print(residues)

{1, 2, 4}


Create the generator corresponding to $g(x)$.

In [7]:
generator = np.array([1 if j in residues else 0 for j in range(1, q+1)])
print(generator)

[1 1 0 1 0 0 0]


Create the generator matrix $P_\textbf{s}$ by applying cyclic shifts to the generator.


In [8]:
P_s = np.array([np.roll(generator, j) for j in range(r)])
print(P_s.T)

[[1 0 0 0]
 [1 1 0 0]
 [0 1 1 0]
 [1 0 1 1]
 [0 1 0 1]
 [0 0 1 0]
 [0 0 0 1]]


### Generate the secret codeword $\textbf{s}$ and hide the generator matrix $P_\textbf{s}$ in a larger matrix $P$.

Append all ones vector to columns of $P_\textbf{s}$.


In [9]:
ones_codeword = [np.ones(q, dtype=int)]
P_s = np.concatenate((ones_codeword, P_s))
print(P_s.T)

[[1 1 0 0 0]
 [1 1 1 0 0]
 [1 0 1 1 0]
 [1 1 0 1 1]
 [1 0 1 0 1]
 [1 0 0 1 0]
 [1 0 0 0 1]]


Initialise secret codeword $\textbf{s} = (1, 0, \dots, 0)$.

In [10]:
s = [1] + [0]*r
print(s)

[1, 0, 0, 0, 0]


Add $q$ random rows, orthogonal to $\textbf{s}$. We ensure orthogonality by padding each row with an extra 0.

In [11]:
orthogonal_rows = [binary_vector(i, r+1) for i in sample(range(1, 2**r), q)]
P = np.column_stack((P_s, *orthogonal_rows))
print(P.T)

[[1 1 0 0 0]
 [1 1 1 0 0]
 [1 0 1 1 0]
 [1 1 0 1 1]
 [1 0 1 0 1]
 [1 0 0 1 0]
 [1 0 0 0 1]
 [0 1 1 1 0]
 [0 0 1 1 1]
 [0 1 1 0 1]
 [0 1 1 1 1]
 [0 0 0 0 1]
 [0 0 1 0 1]
 [0 1 0 0 1]]


Apply random column operations to $P$ to hide the submatrix $P_\textbf{s}$.

In [12]:
for _ in range(3*q):
    # Sample distinct row numbers
    i,j = sample(range(r+1), k=2)
    
    # Add row j to row i
    P[i] = (P[i] + P[j]) % 2
    
    # Adjust s correspondingly
    s[j] = (s[i] + s[j]) % 2 

$P$ is sent to Bob.

In [15]:
print(P.T)

[[0 1 1 0 0]
 [0 1 0 0 1]
 [1 0 1 1 1]
 [1 1 0 0 1]
 [0 0 0 1 0]
 [1 0 0 1 0]
 [0 0 1 1 1]
 [1 1 0 1 1]
 [1 0 0 0 0]
 [0 1 1 1 0]
 [1 1 0 1 0]
 [0 0 0 0 1]
 [0 0 1 0 0]
 [0 1 0 1 1]]


Alice's keeps her secret codeword $\textbf{s}$ safe!

In [14]:
print(s)

[0, 1, 0, 1, 0]
